In [1]:
from utils import authenticate, read_file


screen_names = read_file("screen_names.json")
print(screen_names)

['sehiruniversite', 'sehirmot', 'SehirMYO', 'sehir_iletisim', 'SamSehir', 'SehirSBE', 'isifsehir', 'SehirKariyer', 'SehirClubs', 'sehirlibrary', 'irosehir', 'SehirCHES', 'polssehir', 'hilsehir', 'SehirFelsefe', 'SehirCTV', 'socsehir', 'sehiredebiyat', 'Sehir_Law', 'sehir_cs', 'sehirdramaclub', 'SehirCycling', 'sehirmusicclub', 'sehiricebreaker', 'sehir_fm', 'sehir_munazara', 'SehirKonsey', 'sehirsanat', 'SehirHUKUK', 'sehiryesilay', 'sehirpolitika', 'SehirMuhKulubu', 'gastesehir', 'ieeesehir', 'qbitclub', 'sehircayhane']


In [2]:
api = authenticate()

In [3]:
from crawlers import UserCrawler
from datetime import datetime


users = {}
start_time = datetime.now()
for screen_name in screen_names:
    crawler = UserCrawler(screen_name, api)
    users.update(crawler.crawl(depth=1))
print("Took {} to crawl users".format(datetime.now() - start_time))

Rate limit reached. Sleeping for: 21
Rate limit reached. Sleeping for: 876
Rate limit reached. Sleeping for: 798
Took 0:30:46.701883 to crawl users


Twitter lets us fetch connections of 15 users per 15 minutes

In [4]:
from networks import UserNetwork


incoming = 2
outgoing = 6

network = UserNetwork(users)
network.create()
filtered_users = network.filter(incoming, outgoing)

In [5]:
print("Number of users having {} incoming and {} outgoing edges: {} ".format(incoming, outgoing,len(filtered_users)))

Number of users having 2 incoming and 6 outgoing edges: 225 


In [6]:
from crawlers import UserTweetCrawler


data = {}
start_time = datetime.now()
for user in filtered_users:
    crawler = UserTweetCrawler(api, user)
    data.update(crawler.crawl())
print("Took {} to crawl tweets".format(datetime.now() - start_time))

Rate limit reached. Sleeping for: 54
Rate limit reached. Sleeping for: 502
Rate limit reached. Sleeping for: 502
Rate limit reached. Sleeping for: 510
Rate limit reached. Sleeping for: 516
Rate limit reached. Sleeping for: 529
Rate limit reached. Sleeping for: 501
Took 3:06:36.388060 to crawl tweets


We are allowed to request user's timelines 900 times per 15 minutes and we can fetch 3200 tweets per request

In [7]:
def get_unknowns(data, screen_names):
    unknowns = []
    for user_id in list(data.keys())[:30]:
        user = api.get_user(user_id)
        if user.screen_name not in screen_names:
            unknowns.append(user.id)
    return unknowns


def pretty_print(api, data, user_ids):
    for user_id in user_ids:
        print(api.get_user(user_id).screen_name + ":\n" + "*" * 50)
        tweets = data[user_id]["tweets"]
        if tweets:
            for tweet_id in list(tweets.keys())[:5]:
                print(tweets[tweet_id]["tweet"] + " - " + tweets[tweet_id]["date"] + "\n" + "-" * 50)

In [8]:
user_ids = get_unknowns(data, screen_names)
pretty_print(api, data, user_ids)

SehirDragos:
**************************************************
RT @SehirUniversite: Şimdi yeni bir şehrin var ve bu şehir artık daha da büyük! #GelecekSehirde #SehirDragosta https://t.co/xihcm3MH5I - 2017-05-18 12:22:43
--------------------------------------------------
RT @istmcc: Dragos kampüsü ile birlikte gümbür gümbür gelen bir Şehir Üniversitesi..
(Kadıköy-Tavşantepe metro hattı) https://t.co/p62RcCW5… - 2017-05-18 09:41:54
--------------------------------------------------
RT @peyamicelikcan: Kampüs'te Bisiklet Turu:
İster kendi bisikletinle, ister Üniversite'nin bisikletiyle Kampüs Bisiklet Turu'na katıl... H… - 2017-11-15 19:48:28
--------------------------------------------------
RT @gastesehir: ŞEHİR Dragos Kampüsü Tanıtım Toplantısı https://t.co/YpylE1KmiK - 2017-05-11 12:32:07
--------------------------------------------------
RT @SehirUniversite: ŞEHİR’de değişim başlıyor! Yeni kampüsümüz Dragos bizi bekliyor! https://t.co/3MNe4OUgsP - 2017-05-12 13:31:57
---------------

aysenurkoc5:
**************************************************
RT @ozlembaglar: #kotoncocuklarımızıkullanma duyarli ve bilincli nesiller icin her turlu istismara hayir diyoruz... - 2014-09-19 18:55:28
--------------------------------------------------
RT @SinanCanan: Bunu demiştim, tekrarlıyorum. Hile-hurda-manipülasyon gibi bahanelere sığındıkça siyaset (bile) yapamayacak bazısı... Abre… - 2017-04-16 18:41:53
--------------------------------------------------
RT @stdogann: #WakeUpHumanity | “What is being strewn around in Aleppo today is not the lives or honors of innocent people but the ruins of… - 2016-12-15 19:32:01
--------------------------------------------------
RT @TUGAPonline: Gençlerin her zamn yannda olan ve desteklerini esirgemeyen @halideincekara hocamıza ayrıca teşekkür ederiz.Tecrübelerini n… - 2017-01-14 17:07:16
--------------------------------------------------
RT @Yenisehirlioglu: Meselelere vakıf gençlerle olmak umut verici. @SehirUniversite @timasyayingrubu  #etk

SEHIRGENCTEMA:
**************************************************
Yılın transferini gerçekleştirdik.@myavuzkanar @SEHIRGENCTEMA nın onur üyesi oldu.#sehirkulüpleri https://t.co/mkdMuivliA - 2015-11-05 12:38:20
--------------------------------------------------
RT @TemaVakfi: Toplumsal fayda amaçlayan projeniz var mı?Discover e-volunteering/E-gönüllülüğü Keşfet yarışmasına son başvuru:15 Tem http:/… - 2014-07-12 03:58:58
--------------------------------------------------
RT @temavakfi: Dünyanın su varlığı yaklaşık 1.4 milyar km3’dür.
Su hakkında her şey: https://t.co/0d0VbIFw6b https://t.co/mcl56fSeei - 2016-05-25 20:19:09
--------------------------------------------------
Bayramınızı en içten dileklerimizle kutluyoruz. http://t.co/F8etc5fu4p - 2015-07-17 12:03:35
--------------------------------------------------
RT @TemaVakfi: 21 Mart Dünya Ormancılık Günü ve 22 Mart Dünya Su Gününüz kutlu olsun. http://t.co/PFu5tv3ycF - 2015-03-20 21:21:51
--------------------------------------------